In [1]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from eloguessr import EloGuessr
from utils import load_data, plot_losses

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
data_dir = '/Home/siv33/vbo084/EloGuessr/data/processed'
model_dir = '/Home/siv33/vbo084/EloGuessr/data/processed'

In [3]:
model = torch.load('/Home/siv33/vbo084/EloGuessr/data/processed/' + 'model.pt')

#fnames = ['chess_train_elite_full.pt', 'chess_val_elite_full.pt', 'chess_test_elite_full.pt']
#fnames = ['chess_train_elite_medium.pt', 'chess_val_elite_medium.pt', 'chess_test_elite_medium.pt']
fnames = ['chess_train_both_full.pt', 'chess_val_both_full.pt', 'chess_test_both_full.pt']

BATCH_SIZE = 128
train_dloader, val_dloader, test_dloader = load_data(data_dir, fnames, batch_size=BATCH_SIZE)
del train_dloader

In [4]:
model

EloGuessr(
  (emb_layer): Embedding(6561, 1024, padding_idx=6556)
  (posenc): PositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (linear1): Linear(in_features=1024, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=1024, bias=True)
        (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (dec_block): LinearBlock(
    (fc): Linear(in_features=1024, out_features=1024, bias=True)
    (lnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): 

In [5]:
def eval(model_list: list, dataloader):
    pred_scores = []
    true_scores = []

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)

            scores = model(inputs)
            pred_scores.append(scores.squeeze())  # Ensure scores are in the right shape
            true_scores.append(targets.squeeze())  # Ensure targets are in the right shape


    pred_scores = torch.cat(pred_scores, dim=0)
    true_scores = torch.cat(true_scores, dim=0)
    assert pred_scores.shape == true_scores.shape, f"Shape mismatch: {pred_scores.shape} vs {true_scores.shape}"

    hits = torch.abs(pred_scores - true_scores) <= 50
    hit_percentage = hits.sum().item() / len(hits) * 100

    print(f'Hit Percentage: {hit_percentage:.2f}%')

In [6]:
models = [model]

In [7]:
eval(models, test_dloader)

/opt/conda/conda-bld/pytorch_1712609048481/work/aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [202,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1712609048481/work/aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [202,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1712609048481/work/aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [202,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1712609048481/work/aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [202,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1712609048481/work/aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [202,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/cond

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`